In [1]:
import pygame
from Gameclass import Game, load_game_assets
import maptools
from EntityWidgets import EntityWidget
import EntityWidgets
from SpellWidgets import BowSpell, LinearSpell, CircularSpell, TriangleSpell

class Button:
    def __init__(self, x, y, width, height, text, font, color, text_color, action=None):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.font = font
        self.color = color
        self.text_color = text_color
        self.action = action

    def draw(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        text_surface = self.font.render(self.text, True, self.text_color)
        text_rect = text_surface.get_rect(center=self.rect.center)
        surface.blit(text_surface, text_rect)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Left click
            if self.rect.collidepoint(event.pos):
                if self.action:
                    self.action()

'''
Шаг 1: развёртывание основного окна
'''
pygame.init()
# Получение разрешения экрана
info = pygame.display.Info()  # Возвращает информацию о дисплее
screen_width, screen_height = info.current_w, info.current_h

# Создание окна с точным размером доступного экрана
screen = pygame.display.set_mode((screen_width, screen_height), pygame.RESIZABLE)
pygame.display.set_caption("RaDnDom fight visualiser")

'''
Шаг 2: Подгрузка данных о текущей игре
'''
game = Game()
map_image, players, monsters, num_tiles = load_game_assets(game)

# Рассчитываем параметры для масштабирования изображения карты
screen_width, screen_height = screen.get_size()

left_top, right_bottom, scaled_width, scaled_height = maptools.get_scaled_image_properties(
    map_image, screen_width, screen_height
)
# Создание сетки поверх карты
rows, cols = num_tiles, num_tiles

tile_width = scaled_width // cols
tile_height = scaled_height // rows

corrected_width = tile_width * cols
corrected_height = tile_height * rows

map_image_surface = pygame.image.fromstring(map_image.tobytes(), map_image.size, map_image.mode)
scaled_image_surface = pygame.transform.scale(map_image_surface, (corrected_width, corrected_height))


grid = maptools.create_grid(
    rows, cols, 
    cell_width=scaled_width // cols, 
    cell_height=scaled_height // rows, 
    start_x=left_top[0], 
    start_y=left_top[1],
    total_width=corrected_width,  # Передаем общую ширину изображения
    total_height=corrected_height  # Передаем общую высоту изображения
)

map_manager = maptools.MapManager(grid)

'''
Шаг три: подгружаем виджеты героев и монстров и прописываем их логику
'''

hero_widgets = []
monster_widgets = []

# Диаметр виджетов
widget_diameter = grid[0][0].rect.width 

# Расчет позиций для виджетов
hero_start_x = left_top[0] + corrected_width + 40  # Справа от карты
hero_start_y = left_top[1] + 100
monster_start_x = left_top[0] - widget_diameter - 40  # Слева от карты
monster_start_y = left_top[1] + 100

widget_spacing = widget_diameter + 20
x, y, cell_size = 0, 0, scaled_width // cols
spell_widgets = [BowSpell(screen, x, y, cell_size, map_manager), LinearSpell(screen, x, y, cell_size, map_manager), 
                 CircularSpell(screen, x, y, cell_size, map_manager), TriangleSpell(screen, x, y, cell_size, map_manager)]

# Создаем виджеты для героев и монстров
for i, hero in enumerate(players):
    widget_x = hero_start_x
    widget_y = hero_start_y + i * widget_spacing
    hero_widgets.append(EntityWidget(hero, widget_diameter, widget_x, widget_y, widget_diameter, map_manager, screen, spell_widgets))

for i, monster in enumerate(monsters):
    widget_x = monster_start_x
    widget_y = monster_start_y + i * widget_spacing
    monster_widgets.append(EntityWidget(monster, widget_diameter, widget_x, widget_y, widget_diameter, map_manager, screen, spell_widgets))

# Объединяем в 1 список для обработки
all_widgets = hero_widgets + monster_widgets + spell_widgets
entity_widgets = all_widgets

def recalculate_damage():
    # Все EntityWidgets выполняют метод get_damage
    for widget in entity_widgets:
        if isinstance(widget, EntityWidget):
            widget.get_damage()

    # Обновление экрана
    pygame.display.flip()

    # Сброс урона в MapManager
    map_manager.reset_damage()


'''
Шаг последний: основной цикл программы
'''
font = pygame.font.Font(None, 36)
button_width, button_height = 200, 50
button_x = screen_width - button_width - 20
button_y = screen_height - button_height - 20

recalculate_button = Button(
    x=button_x, 
    y=button_y, 
    width=button_width, 
    height=button_height, 
    text="Recalc Damage", 
    font=font, 
    color=(0, 128, 0), 
    text_color=(255, 255, 255), 
    action=recalculate_damage
)

running = True
while running:
    mouse_pos = pygame.mouse.get_pos()  # Позиция мыши

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Обработка событий кнопки
        recalculate_button.handle_event(event)

        # Обработка событий виджетов
        for widget in entity_widgets:
            widget.handle_event(event)

    # Отрисовка карты
    screen.fill((100, 100, 100))  # Заливка фона темно-серым
    screen.blit(scaled_image_surface, left_top)

    # Отрисовка сетки
    for row in grid:
        for cell in row:
            if cell.is_hovered(mouse_pos):
                cell.draw(screen, border_color=(255, 255, 255), fill_color=(255, 255, 255, 80))
            else:
                cell.draw(screen, border_color=(255, 255, 255), fill_color=(0, 0, 0, 50))

    # Отрисовка виджетов
    for widget in all_widgets:
        if isinstance(widget, BowSpell) and widget.visible:  
            mx, my = pygame.mouse.get_pos()
            widget.draw(mx, my)
        elif isinstance(widget, LinearSpell) and widget.visible:  
            mx, my = pygame.mouse.get_pos()
            widget.draw(mx, my)
        elif isinstance(widget, TriangleSpell) and widget.visible:  
            mx, my = pygame.mouse.get_pos()
            widget.draw(mx, my)
        elif isinstance(widget, CircularSpell) and widget.visible:  
            mx, my = pygame.mouse.get_pos()
            widget.draw(mx, my)
        else:
            try:
                widget.draw(screen)
            except TypeError:
                ...

    # Отрисовка кнопки
    recalculate_button.draw(screen)

    pygame.display.flip()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyError: 'death_avatar'

In [ ]:
import pygame
from Gameclass import Game, load_game_assets
import maptools
from EntityWidgets import EntityWidget
import EntityWidgets
from SpellWidgets import BowSpell, LinearSpell, CircularSpell, TriangleSpell

'''
Шаг 1: развёртывание основного окна
'''
pygame.init()
# Получение разрешения экрана
info = pygame.display.Info()  # Возвращает информацию о дисплее
screen_width, screen_height = info.current_w, info.current_h

# Создание окна с точным размером доступного экрана
screen = pygame.display.set_mode((screen_width, screen_height), pygame.RESIZABLE)
pygame.display.set_caption("RaDnDom fight visualiser")

'''
Шаг 2: Подгрузка данных о текущей игре
'''
game = Game()
map_image, players, monsters, num_tiles = load_game_assets(game)

# Рассчитываем параметры для масштабирования изображения карты
screen_width, screen_height = screen.get_size()

left_top, right_bottom, scaled_width, scaled_height = maptools.get_scaled_image_properties(
    map_image, screen_width, screen_height
)
# Создание сетки поверх карты
rows, cols = num_tiles, num_tiles

tile_width = scaled_width // cols
tile_height = scaled_height // rows

corrected_width = tile_width * cols
corrected_height = tile_height * rows

map_image_surface = pygame.image.fromstring(map_image.tobytes(), map_image.size, map_image.mode)
scaled_image_surface = pygame.transform.scale(map_image_surface, (corrected_width, corrected_height))


grid = maptools.create_grid(
    rows, cols, 
    cell_width=scaled_width // cols, 
    cell_height=scaled_height // rows, 
    start_x=left_top[0], 
    start_y=left_top[1],
    total_width=corrected_width,  # Передаем общую ширину изображения
    total_height=corrected_height  # Передаем общую высоту изображения
)
#print(f'x0 = {left_top[0]}, y0 = {left_top[1]}, sw = {corrected_width // cols}, sh = {corrected_height // rows}')

map_manager = maptools.MapManager(grid)

'''
Шаг три: подгружаем виджеты героев и монстров и прописываем их логику
'''

hero_widgets = []
monster_widgets = []

# Диаметр виджетов
widget_diameter = grid[0][0].rect.width 

# Расчет позиций для виджетов
hero_start_x = left_top[0] + corrected_width + 40  # Справа от карты
hero_start_y = left_top[1] + 100
monster_start_x = left_top[0] - widget_diameter - 40  # Слева от карты
monster_start_y = left_top[1] + 100

widget_spacing = widget_diameter + 20
x, y, cell_size = 0, 0, scaled_width // cols
spell_widgets = [BowSpell(screen, x, y, cell_size, map_manager), LinearSpell(screen, x, y, cell_size, map_manager), 
                 CircularSpell(screen, x, y, cell_size, map_manager), TriangleSpell(screen, x, y, cell_size, map_manager)]



# Создаем виджеты для героев и монстров
for i, hero in enumerate(players):
    widget_x = hero_start_x
    widget_y = hero_start_y + i * widget_spacing
    hero_widgets.append(EntityWidget(hero, widget_diameter, widget_x, widget_y, widget_diameter, map_manager, screen, spell_widgets))

for i, monster in enumerate(monsters):
    widget_x = monster_start_x
    widget_y = monster_start_y + i * widget_spacing
    monster_widgets.append(EntityWidget(monster, widget_diameter, widget_x, widget_y, widget_diameter, map_manager, screen, spell_widgets))
#объединяем в 1 список для обработки
'''
В этом месте есть шанс сломать вообще всё
'''
all_widgets = hero_widgets + monster_widgets + spell_widgets
entity_widgets = all_widgets

def get_cell_under_widget(widget, grid):
    widget_center_x = widget.x + widget.diameter / 2
    widget_center_y = widget.y + widget.diameter / 2

    for row in grid:
        for cell in row:
            # Проверяем, находится ли центр виджета в пределах клетки
            if (cell.rect.x <= widget_center_x <= cell.rect.x + cell.rect.width and
                cell.rect.y <= widget_center_y <= cell.rect.y + cell.rect.height):
                return cell
    return None


'''
Шаг последний: основной цикл программы
'''
running = True
while running:
    mouse_pos = pygame.mouse.get_pos()  # Позиция мыши

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Обработка событий виджетов
        for widget in entity_widgets:
            widget.handle_event(event)

    # Отрисовка карты
    screen.fill((100, 100, 100))  # Заливка фона темно-серым
    screen.blit(scaled_image_surface, left_top)

    # Отрисовка сетки
    for row in grid:
        for cell in row:
            if cell.is_hovered(mouse_pos):
                cell.draw(screen, border_color=(255, 255, 255), fill_color=(255, 255, 255, 80))
            else:
                cell.draw(screen, border_color=(255, 255, 255), fill_color=(0, 0, 0, 50))

    # Отрисовка виджетов
    for widget in all_widgets:
        try:
            widget.draw(screen)
        except TypeError:
            widget.draw()
        
    pygame.display.flip()

pygame.quit()

In [ ]:
import pygame
import json
from pygame.locals import *
from entity_widgets import EntityWidget, load_entities_from_json

# Инициализация Pygame
pygame.init()

# Цвета
WHITE = (255, 255, 255)

# Загружаем карту
MAP_IMAGE_PATH = "map.png"  # Укажите путь к своему изображению карты
map_image = pygame.image.load(MAP_IMAGE_PATH)
map_width, map_height = map_image.get_size()  # Получаем размеры карты

# Основные настройки окна
screen_width = map_width + 2 * 150  # Добавляем боковые панели для сущностей
screen_height = map_height

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("DnD Map with Characters")

# Загружаем сущности из JSON
entities = load_entities_from_json("entities.json")  # Укажите путь к своему JSON

# Разделяем сущности на игроков и монстров
players = [e for e in entities if e.entity_type == "Player"]
monsters = [e for e in entities if e.entity_type == "Monster"]

# Теперь создаём виджеты сущностей, передавая map_position
player_widgets = [EntityWidget(player, screen_width - 130, 50 + idx * 60, map_manager.map_position) for idx, player in enumerate(players)]
monster_widgets = [EntityWidget(monster, 30, 50 + idx * 60, map_manager.map_position) for idx, monster in enumerate(monsters)]

# Главный цикл игры
running = True
while running:
    screen.fill(WHITE)

    # Рисуем карту
    screen.blit(map_image, (150, 0))  # Располагаем карту в центре

    # Рисуем виджеты для сущностей
    for widget in player_widgets + monster_widgets:
        widget.draw()

    pygame.display.flip()

    # Обработка событий
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Закрытие Pygame
pygame.quit()


In [ ]:
UTILS

In [ ]:
import tkinter as tk
from PIL import ImageTk, Image, ImageDraw
from MapTiler import open_image_and_create_grid
from EntityManager import load_entities
from SidebarManager import add_sidebars

def main():
    # Создаем главное окно
    root = tk.Tk()

    # Загружаем карту и создаем сетку
    map_image, num_cells_x, num_cells_y = open_image_and_create_grid(root)  # Передаем root в функцию

    if map_image is None:
        return  # Если карта не загружена, завершаем выполнение

    # Преобразуем изображение карты в формат, который можно использовать в Tkinter
    map_image_tk = ImageTk.PhotoImage(map_image)

    # Вычисляем размер клетки (на основе карты и количества тайлов)
    tile_size = map_image.width // num_cells_x  # Берем ширину карты и делим на количество клеток по горизонтали

    # Пример использования: отображение карты на холсте
    canvas = tk.Canvas(root, width=map_image.width, height=map_image.height)
    canvas.create_image(0, 0, anchor=tk.NW, image=map_image_tk)
    canvas.pack(side=tk.LEFT)

    # Сохраняем ссылку на изображение, чтобы избежать его удаления
    canvas.image = map_image_tk

    # Загружаем сущности (например, игроки)
    players = load_entities("player.json", root=root)
    #print(f'Players: {[vars(player) for player in players]}')

    monsters = load_entities("monster.json", root=root)
    #print(f'Monsters: {[vars(monster) for monster in monsters]}')
    # Запускаем главный цикл приложения

        # Добавляем боковые панели
    circle_size = 50  # Задаем размер кружков
    add_sidebars(root, canvas, players, monsters, circle_size)

    #print(num_cells_x, num_cells_y)
    root.mainloop()


if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
from tkinter import filedialog, simpledialog
from PIL import Image, ImageTk

def get_grid_dimensions():
    # Создаем новое окно для ввода данных
    input_window = tk.Toplevel()
    input_window.title("Grid Input")

    # Метка и поле ввода для X
    tk.Label(input_window, text="Cells along X axis:").grid(row=0, column=0, padx=10, pady=5)
    x_entry = tk.Entry(input_window)
    x_entry.grid(row=0, column=1, padx=10, pady=5)

    # Метка и поле ввода для Y
    tk.Label(input_window, text="Cells along Y axis:").grid(row=1, column=0, padx=10, pady=5)
    y_entry = tk.Entry(input_window)
    y_entry.grid(row=1, column=1, padx=10, pady=5)

    # Переменные для сохранения результата
    dimensions = {"x": None, "y": None}

    # Функция для обработки нажатия кнопки "OK"
    def submit():
        try:
            dimensions["x"] = int(x_entry.get())
            dimensions["y"] = int(y_entry.get())
            input_window.destroy()  # Закрываем окно после ввода данных
        except ValueError:
            tk.messagebox.showerror("Invalid Input", "Please enter valid integers!")

    # Кнопка для подтверждения ввода
    tk.Button(input_window, text="OK", command=submit).grid(row=2, column=0, columnspan=2, pady=10)

    # Ожидаем закрытия окна
    input_window.transient()  # Устанавливаем окно поверх главного
    input_window.grab_set()  # Перехватываем фокус ввода
    input_window.wait_window()  # Ждем закрытия окна

    return dimensions["x"], dimensions["y"]


def open_image_and_create_grid(root=None):
    # Открытие окна выбора файла
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpeg;*.jpg")])
    if not file_path:
        return None

    num_cells_x, num_cells_y = get_grid_dimensions()
    
    if not num_cells_x or not num_cells_y:
        return None

    # Загрузка изображения
    img = Image.open(file_path)
    img_width, img_height = img.size

    # Проверяем, что root передан, если нет — создаём его
    if root is None:
        root = tk.Tk()
        root.title("DnD Map Viewer")

    # Создание холста (Canvas) для отображения изображения
    canvas = tk.Canvas(root, width=img_width, height=img_height)
    canvas.pack()

    # Конвертация изображения в формат Tkinter
    img_tk = ImageTk.PhotoImage(img)
    canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)

    # Запись изображения, чтобы не потерять ссылку на него
    canvas.image = img_tk  # Это нужно для предотвращения удаления изображения сборщиком мусора

    # Вычисление размеров клеток
    cell_width = img_width / num_cells_x
    cell_height = img_height / num_cells_y

    # Функция подсветки клетки
    def highlight_cell(event):
        x, y = event.x, event.y
        cell_x = int(x // cell_width)
        cell_y = int(y // cell_height)

        # Очистка Canvas от предыдущих подсветок
        canvas.delete("highlight")

        # Рисование подсветки
        canvas.create_rectangle(
            cell_x * cell_width, cell_y * cell_height,
            (cell_x + 1) * cell_width, (cell_y + 1) * cell_height,
            outline="white", width=2, tags="highlight"
        )

    # Привязка события для подсветки клетки
    canvas.bind("<Motion>", highlight_cell)

    # Возвращаем изображение для дальнейшего использования
    return img, num_cells_x, num_cells_y

# Запуск основного функционала
if __name__ == "__main__":
    open_image_and_create_grid()